In [10]:
import json
from math import sin, cos, pi
import numpy as np

In [11]:
with open('isd.isd', 'r') as f:
    isd = json.load(f)

In [13]:
isd

{'boresight': [0.0, 0.0, 1.0],
 'ccd_center': 512.5,
 'ephemeris_time': 418855170.49264956,
 'focal_length': 549.1178195372703,
 'focal_length_epsilon': 0.5,
 'ifov': 25.44,
 'instrument_id': 'MDIS-NAC',
 'itrans_line': [0.0, 0.0, 71.42857143],
 'itrans_sample': [0.0, 71.42857143, 0.0],
 'kappa': -0.9630375478615623,
 'nlines': 1024,
 'nsamples': 1024,
 'odt_x': [0.0,
  1.001854269623802,
  0.0,
  0.0,
  -0.0005094440474941111,
  0.0,
  1.004010471468856e-05,
  0.0,
  1.004010471468856e-05],
 'odt_y': [0.0,
  0.0,
  1.0,
  0.0009060010594996751,
  0.0,
  0.0003574842626620758,
  0.0,
  1.004010471468856e-05,
  0.0],
 'omega': 2.256130940792258,
 'original_half_lines': 512.0,
 'original_half_samples': 512.0,
 'phi': 0.09433201631102328,
 'pixel_pitch': 0.014,
 'semi_major_axis': 2439.4,
 'semi_minor_axis': 2439.4,
 'spacecraft_name': 'Messenger',
 'starting_detector_line': 1.0,
 'starting_detector_sample': 9.0,
 'target_name': 'Mercury',
 'transx': [0.0, 0.014, 0.0],
 'transy': [0.0, 0.

In [14]:
# 512, 512 are the focal width/height in pixels divided by 2
a = np.zeros((3,3))
a[0,0] = isd['focal_length'] 
a[1,1] = isd['focal_length'] 
a[:,2] = [isd['original_half_samples'],isd['original_half_lines'],1]
a

array([[ 549.11781954,    0.        ,  512.        ],
       [   0.        ,  549.11781954,  512.        ],
       [   0.        ,    0.        ,    1.        ]])

In [54]:
def rotation_from_opk(o, p, k):
    om = np.empty((3,3))
    om[:,0] = [1,0,0]
    om[:,1] = [0, cos(o), -sin(o)]
    om[:,2] = [0, sin(o), cos(o)]
    
    pm = np.empty((3,3))
    pm[:,0] = [cos(p), 0, sin(p)]
    pm[:,1] = [0,1,0]
    pm[:,2] = [-sin(p), 0, cos(p)]
    
    km = np.empty((3,3))
    km[:,0] = [cos(k), -sin(k), 0]
    km[:,1] = [sin(k), cos(k), 0]
    km[:,2] = [0,0,1]
    
    return km.dot(pm).dot(om)

# This makes a great test case (Mikhail p.95 has the rotation matrix.)
o = isd['omega']
p = isd['phi']
k = isd['kappa']
camera_rotation = rotation_from_opk(o, p, k)

def setfocalrot(x, y, z):    
    # This is a focal plan rotation matrix that is flipping the camera vertically (I think)
    # 0,0,1000 is the z position of the spacecraft
    c = np.zeros((3,4))
    c[0,0] = 1
    c[1,1] = -1
    c[2,2] = -1
    c[:,3] = [x,y,z]
    return c

# Arguments are spacecraft position: x, y, z
c = setfocalrot(isd['x_sensor_origin'],
                isd['y_sensor_origin'],
                isd['z_sensor_origin'])

def pixelloc(a,b,c, pos, tx, ty):
    res = a.dot(b).dot(c)
    res = res.dot(pos)
    res[0] /= res[-1]
    res[1] /= res[-1]
    res[2] /= res[-1]
    # Mapping from focal plane to pixel space
    res[0] *= tx
    res[1] *= ty
    return res[:2]
    
# pixel position on the surface: x,y,z,1
"""position = np.array([1116890,
                     -1604470,
                     1459570,
                     1])"""

position = np.array([1131980,
                    -1597990,
                    1455060,
                    1])

ploc = pixelloc(a, camera_rotation, c, position, isd['transx'][1], isd['transy'][2])
ploc

array([ 36.87751071,  92.45929751])

In [53]:
ploc

array([ 36.87751071,  92.45929751])

In [35]:
ploc[1] - 101

-7.823753891887506